In [1]:
from vtk import vtkUnstructuredGridReader
from vtk import vtkXMLUnstructuredGridReader
from vtk.util import numpy_support as VN
from vtk.util.numpy_support import vtk_to_numpy
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
# A python code that produces an image of the flexing of an embedded cantilever and compares it to the
# Analytic solution from Geodynamics 3rd Edition by Turcotte and Schubert, equation 3.85
# This code requires that vtk is installed on the computer
# running the code, and is designed only for when ASPECT outputs into ONE .vtu file per output. This is 
# turned on by adding the following line to the subsection Visualization in the input file:

#  set Number of grouped files = 1

# The Arguments are:
#    file_path - the pathway to the file you want the code to open
#    composition__name - what the composition that makes up the beam in ASPECT is called
#    drho - the density difference between the beam and the surrounding material
#    g - the gravity used in ASPECT, the analytic solutions required that g be negative
#    L - The length of the beam
#    mu - the elastic shear moulus used in the input file
#    nu - Poiossons ratio, 0.5 if incompressible
#    h - the thickness of the beam
############################################################################################################
def beam_image(file_path, composition_name, drho, g, L, mu, nu, h):
    # These lines are reading in the vtu file and creating numpy arrays
    reader = vtkXMLUnstructuredGridReader()
    reader.SetFileName(file_path)
    reader.Update()
    data = reader.GetOutput()
    points = data.GetPoints()
    x = vtk_to_numpy(points.GetData())
    comp = vtk_to_numpy(data.GetPointData().GetArray(composition_name))
    
    # this pulls out the x,y points where the composition is set to the 'beam' value
    beam_location = []
    for n in range(len(comp)):
        if comp[n] > 0.5:
            beam_location.append(x[n])
    beam_location = np.array(beam_location)
    beamx = beam_location[:, 0] # x values of beam
    beamy = beam_location[:, 1] # y values of beam
    # additional parameters for analytic
    E = 2 * mu * (1 + nu)
    D = E * h**3 / (12 * (1 - nu**2))
    q = drho * g * h
    xaxis = np.linspace(0, L, len(beam_location))
    ymax = max(beamy)
    ymin = min(beamy)
    
    # Calculate the midpoint of end of Beam
    topx = []
    botx = []
    topy = []
    boty = []
    for i in range(len(beamx)):
        if beamy[i] == ymin and beamx[i] > 0.95 * L:
            botx.append(beamx[i])
            boty.append(beamy[i])
        if beamy[i] == ymin + h and beamx[i] > 0.95 * L:
            topx.append(beamx[i])
            topy.append(beamy[i])
    topx = max(topx)
    topy = topy[0]
    botx = max(botx)
    boty = boty[0]
    midx = abs(topx - botx) / 2 + botx
    midy = abs(topy - boty) / 2 + boty
    
    # Calculate the Deviation between ASPECT Midpoint and Analytic Midpoint
    mid_analytic = q / (24 * D) * midx**2 * (midx**2 - 4*L*midx + 6*L**2)
    mid_err = abs(mid_analytic - (midy - ymax + h/2)) / abs(mid_analytic) * 100
    
    # analytic solution
    ytop = q / (24 * D) * xaxis**2 * (xaxis**2 - 4*L*xaxis + 6*L**2) + ymax
    ymid = q / (24 * D) * xaxis**2 * (xaxis**2 - 4*L*xaxis + 6*L**2) + ymax - h/2
    ybot = q / (24 * D) * xaxis**2 * (xaxis**2 - 4*L*xaxis + 6*L**2) + ymax - h
    # plot
    plt.figure(dpi = 140)
    plt.plot(xaxis / 1e3, ytop / 1e3, c = 'b')
    plt.plot(xaxis / 1e3, ymid / 1e3, c = 'b')
    plt.plot(xaxis / 1e3, ybot / 1e3, c = 'b', label = 'Analytic')
    plt.scatter(beamx / 1e3, beamy / 1e3, s = 5, c = 'r', label = "ASPECT")
    plt.scatter(midx / 1e3, midy / 1e3)
    plt.scatter(topx / 1e3, topy / 1e3, c = 'k')
    plt.scatter(botx / 1e3, boty / 1e3, c = 'k')
    plt.xlabel('x - km')
    plt.ylabel('y - km')
    plt.legend()
    print("The % error between the model midpoint and the analytic midpoint is " + str(mid_err))